# Querying for Potential Redox-Regulated Enzymes

## Installations

Install Jmol with: `sudo apt install -y jmol`

## Download 1FRF and 1FRV

In [ ]:
wget 'https://files.rcsb.org/download/1FRF.pdb'

In [ ]:
wget 'https://files.rcsb.org/download/1FRV.pdb'

# Visual Inspection

Run `jmol` and use the following commands:
```
select all
spacefill 50
select cys and sulfur
color yellow
spacefill 250
monitor 3914 5273
```

## Computational Inspection
Requires the following script:
```
# save as distance.awk
# searches close cysteine sulfur atoms in a structure
# requires a structure file (*.pdb)
# usage: awk -f distance.awk structure.pdb

BEGIN{print "Cysteines in the Structure..."; ORS=""}

$1=="ATOM" && $4=="CYS" && $3=="SG" {
print $4$6", "
cys_x[$6]=$7; cys_y[$6]=$8; cys_z[$6]=$9
}

END{ ORS="\n"
for (key1 in cys_x) {
  for (key2 in cys_x) { 
      dx=cys_x[key1]-cys_x[key2]
      dy=cys_y[key1]-cys_y[key2]
      dz=cys_z[key1]-cys_z[key2]
      distance=sqrt(dx^2+dy^2+dz^2)
      if (distance < 3 && distance != 0 && key1<key2) {
        i++
        candidate[i]=key1"-"key2": "distance
      }
  }
}
print "\nCandidates ..."
for (keys in candidate) {print candidate[keys]}
}
```

In [ ]:
awk -f distance.awk 1FRF.pdb

In [ ]:
awk -f distance.awk 1FRV.pdb

Run in Jmol
```
load =1FRF
hide water                    # remove water molecules
spacefill off                 # reduce atom spheres
select cys436.sg, cys259.sg, cys75.sg, cys546.sg   # select SG atoms
spacefill 300
```

### Download PDB IDs
Download all IDs as described in the book with the following parameters:
- were derived from \textit{Escherichia} (either directly or via over-expression)
- have an X-ray resolution less than 2 Angström
- consist of one single protein chain

Save the IDs as comma-separated list in *ids.txt*.

### Download PDB Script

In [ ]:
wget "https://www.rcsb.org/scripts/batch_download.sh"

In [ ]:
chmod u+x batch_download.sh

In [ ]:
mkdir EcoliStructures

In [ ]:
cut -c 1-499 ids.txt > ids-100.txt # extract first 100 IDs

In [ ]:
./batch_download.sh -f ids-100.txt -o EcoliStructures -p

In [ ]:
for i in $(sed 's/,/ /g' ids-100.txt); do wget -P EcoliStructures2 "https://files.rcsb.org/download/$i.pdb" ; done

### Batch Analysis

In [ ]:
for i in EcoliStructures2/*.pdb; do awk -f distance-batch.awk $i; done

In [ ]:
for i in *.pdb; do awk -f distance-batch.awk $i; done

### Open in Jmol
Use the command `jmol -s EcoliStructures2/1B8J.pdb.script` to view a structure in Jmol.